In [ ]:
# https://community.openai.com/t/error-with-openai-1-56-0-client-init-got-an-unexpected-keyword-argument-proxies/1040332/11
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

In [2]:
def prompt_for_generating_empathetic_answer():
    return {"role": "system", "content": """
You are refining medical responses to be more empathetic while maintaining accuracy.

Important: DO NOT add greetings like "Hi there!" or "Hello!" since this may be part of an ongoing conversation.

Guidelines:
1. Start directly by acknowledging the specific concern mentioned
2. Use simple, clear language
3. Add 1-2 appropriate emojis (😊💙🌟 etc.)
4. End with supportive, encouraging words
5. Keep medical accuracy intact
6. Always recommend consulting healthcare professionals

Transform the response to be warmer and more caring while addressing the medical question directly.

Example:
Instead of: "Hi there! 😊 I understand your concerns about diabetes..."
Use: "I understand your concerns about diabetes... 😊"
"""}

In [ ]:
OPENAI_API_KEY = "your own key"

In [4]:
import time
from openai import OpenAI

def make_answer_empathetic(original_answer, max_retries=3):
    client = OpenAI(api_key=OPENAI_API_KEY)

    msgs = [prompt_for_generating_empathetic_answer()]
    msgs.append({"role": "user", "content": original_answer})

    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=msgs,
                temperature=0.3,
                max_tokens=500  # Add token limit to control length
            )
            return completion.choices[0].message.content

        except Exception as e:
            if attempt < max_retries - 1:
                print(f"API call failed (attempt {attempt + 1}), retrying in 2 seconds...")
                time.sleep(2)
            else:
                print(f"All API attempts failed: {e}")
                return original_answer  # Fallback to original

In [5]:
def process_message(message_pair, tokenizer=None):

    prompt   = message_pair[1]["content"]   # user question
    rejected = message_pair[2]["content"]   # original answer

    chosen = make_answer_empathetic(rejected)

    print(chosen[:100])

    return {
        "prompt":   prompt,
        "chosen":   chosen,
        "rejected": rejected
    }

# load data and generate with empathy

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install datasets

In [6]:
from datasets import load_dataset

# Add sample size control
SAMPLE_SIZE = 1000  # Adjust this number as needed

dataset = load_dataset("json", data_files="sft_dataset.json")
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(min(SAMPLE_SIZE, len(dataset["train"]))))

print(f"Using {len(dataset['train'])} samples for DPO generation")
print(dataset)

D:\soft\python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using 1000 samples for DPO generation
DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 1000
    })
})


In [7]:
dataset["train"][0]

{'messages': [{'role': 'system',
   'content': "You are a highly professional medical assistant. Your task is to provide accurate and well-structured medical information based on a patient's symptoms. Your tone must be objective and professional. Always start with a polite greeting and end with a supportive, yet professional, closing. When discussing possible diagnoses or treatments, it is critical to state that this is for informational purposes only and strongly recommend a consultation with a healthcare professional."},
  {'role': 'user',
   'content': 'I have been having alot of catching ,pain and discomfort under my right rib.  If I twist to either side especially my right it feels like my rib actually catches on something and at times I have to stop try to catch my breath and wait for it to subside.  There are times if I am laughing too hard that it will do the same thing but normally its more so if I have twisted or moved  a certain way'},
  {'role': 'assistant',
   'content': '

In [8]:
from tqdm import tqdm
import time
import json

start_time = time.time()
dpo_dataset = []
failed_count = 0

print(f"Starting DPO dataset generation for {len(dataset['train'])} samples...")

for idx, message_pair in enumerate(tqdm(dataset["train"]["messages"], desc="Generating empathetic responses")):
    try:
        result = process_message(message_pair)
        dpo_dataset.append(result)

        # Show progress every 50 samples
        if (idx + 1) % 50 == 0:
            elapsed = time.time() - start_time
            rate = (idx + 1) / elapsed * 60  # samples per minute
            print(f"Progress: {idx + 1}/{len(dataset['train'])} | Rate: {rate:.1f} samples/min | Failures: {failed_count}")

    except Exception as e:
        failed_count += 1
        print(f"Failed to process sample {idx}: {e}")
        continue

with open("./dpo_dataset.json", "w", encoding="utf-8") as f:
    json.dump(dpo_dataset, f, ensure_ascii=False, indent=2)

print(f"DPO Dataset Saved! Time taken: {(time.time() - start_time)/60:.1f} minutes")
print(f"Successfully generated: {len(dpo_dataset)}/{len(dataset['train'])} samples")
print(f"Failed: {failed_count} samples")

Starting DPO dataset generation for 1000 samples...


Generating empathetic responses:   0%|          | 1/1000 [00:05<1:30:41,  5.45s/it]

It sounds like you're experiencing pain when turning to one side, which can be quite uncomfortable. 


Generating empathetic responses:   0%|          | 2/1000 [00:08<1:05:41,  3.95s/it]

I understand your concerns about the painful and swollen lumps on your labium. It sounds like you ma


...

It sounds like you're dealing with contact dermatitis from using Derive BPO gel, and that can be qui


Generating empathetic responses: 100%|█████████▉| 999/1000 [52:24<00:03,  3.84s/it]

It sounds like you're experiencing some discomfort that could be related to kidney stones or hemorrh


Generating empathetic responses: 100%|██████████| 1000/1000 [52:27<00:00,  3.15s/it]

I understand you're seeking clarity on spinal cord injuries, particularly at the C4 level. 😊 You're 
Progress: 1000/1000 | Rate: 19.1 samples/min | Failures: 0
DPO Dataset Saved! Time taken: 52.5 minutes
Successfully generated: 1000/1000 samples
Failed: 0 samples


result

In [3]:
from datasets import load_dataset

dpo_dataset = load_dataset("json", data_files="./dpo_dataset.json")
print(dpo_dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 1000
    })
})


In [2]:
dpo_dataset["train"][0]

{'prompt': 'I have been having alot of catching ,pain and discomfort under my right rib.  If I twist to either side especially my right it feels like my rib actually catches on something and at times I have to stop try to catch my breath and wait for it to subside.  There are times if I am laughing too hard that it will do the same thing but normally its more so if I have twisted or moved  a certain way',
 'chosen': "It sounds like you're experiencing pain when turning to one side, which can be quite uncomfortable. This could likely be due to a moderate muscular strain, possibly from lifting something heavy or during certain activities. 😊 Taking a simple pain reliever and getting plenty of rest can help. Sleeping on your back might also provide some relief.\n\nIf your ribs are tender to touch, it could be helpful to consider getting an X-ray to rule out any other issues. Additionally, if you have symptoms like a cough, cold, or sore throat, it’s important to check for respiratory infec

# Further Optimization

In [ ]:
from huggingface_hub import login

HF_Token = "your own key"
login(token=HF_Token)

In [2]:
# !pip install hf_xet
!pip install huggingface_hub[hf_xet]


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 3B
tokenizer_3B = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct"
)
Llama_3B_Instruct = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    torch_dtype=torch.float32
)

D:\soft\python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
D:\soft\python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\23104\.cache\huggingface\hub\models--meta-llama--Llama-3.2-3B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DI

In [4]:
len(dpo_dataset["train"])

dpo_data = list()

for idx in range(len(dpo_dataset["train"])):

  tmp_prompt = dpo_dataset["train"]["prompt"][idx]
  tmp_chosen = dpo_dataset["train"]["chosen"][idx]
  tmp_reject = dpo_dataset["train"]["rejected"][idx]

  msg_prompt = [{"role": "system", "content": "You are a compassionate medical assistant who responds with empathy and care. Use emojis appropriately 😊💙"}, {"role": "user", "content": tmp_prompt}]
  msg_chosen = [{"role": "assistant", "content": tmp_chosen}]
  msg_reject = [{"role": "assistant", "content": tmp_reject}]

  dpo_data.append({
    "prompt": tokenizer_3B.apply_chat_template(msg_prompt, tokenize=False),
    "chosen": tokenizer_3B.apply_chat_template(msg_chosen, tokenize=False),
    "rejected": tokenizer_3B.apply_chat_template(msg_reject, tokenize=False)
  })

import json
with open("./dpo_dataset_V2.json", "w", encoding="utf-8") as f:
    json.dump(dpo_data, f, ensure_ascii=False, indent=4)

In [5]:
from datasets import load_dataset

dpo_dataset_V2 = load_dataset("json", data_files="./dpo_dataset_V2.json")
print(dpo_dataset_V2)

Generating train split: 1000 examples [00:00, 22597.77 examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 1000
    })
})


In [6]:
dpo_dataset_V2["train"][0]

{'prompt': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 07 Aug 2025\n\nYou are a compassionate medical assistant who responds with empathy and care. Use emojis appropriately 😊💙<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI have been having alot of catching ,pain and discomfort under my right rib.  If I twist to either side especially my right it feels like my rib actually catches on something and at times I have to stop try to catch my breath and wait for it to subside.  There are times if I am laughing too hard that it will do the same thing but normally its more so if I have twisted or moved  a certain way<|eot_id|>',
 'chosen': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 07 Aug 2025\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIt sounds like you're experiencing pain when turning to one side, which can be quite uncomfor